In [1]:
import torch
import os
import mlflow
from PIL import Image
from torchvision import datasets
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import Compose
import onnxruntime
import json

In [2]:
# Dirs
cwd= os.getcwd()
model_uri= os.path.join(cwd,"../mlruns/0/models/m-e6eff4b8933a4b528f309d0afbc4474b/artifacts/")
loaded_model= mlflow.pytorch.load_model(model_uri)
data_dir= os.path.join(cwd, "../data")
artifacts_dir= os.path.join(cwd, "../artifacts")
mlruns_dir= os.path.join(cwd, "../mlruns")


2025/09/22 17:57:29 WARNING mlflow.pytorch: Stored model version '2.8.0+cu126' does not match installed PyTorch version '2.5.1+cu121'


In [3]:
dataset= datasets.CIFAR100(root= data_dir, download= True, train= True)
class_names= dataset.classes

Files already downloaded and verified


In [4]:
def pytorch_inference(model, image_path, transforms, class_names, device):
    try:
        img= Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return
    
    input_tensor= transforms(img)
    input_batch= input_tensor.unsqueeze(0)
    input_batch= input_batch.to(device)

    model.eval()
    with torch.no_grad():
        output= model(input_batch)
    
    probabilities= F.softmax(output, dim= 1)
    confidence, predicted_idx= torch.max(probabilities, 1)
    predicted_class_name= class_names[predicted_idx.item()]
    confidence_score= confidence.item()
    print(f"Predicted Class: {predicted_class_name}")
    print(f"Confidence: {confidence_score:.4f}")
    return input_batch, output
image_path= os.path.join(data_dir, "test/test6.jpeg")
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform= transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor, torch_output= pytorch_inference(loaded_model, image_path, transform, class_names, device)

Predicted Class: cockroach
Confidence: 0.5041


In [5]:
loaded_model.eval()
onnx_model_path= os.path.join(artifacts_dir, "resnet18_fft_dynamic.onnx")
torch.onnx.export(
    loaded_model,
    input_tensor,
    onnx_model_path,
    input_names=['x'],  
    output_names=['output'],  # Standard for classifier logits
    dynamic_axes={
        'x': {0: 'batch_size'},  # Makes batch dim (index 0) dynamic (1-N)
        'output': {0: 'batch_size'}  # Also dynamic for output
    },
    opset_version=11,  # Stable for ResNet18 (use 13+ if PyTorch 1.8+ for newer ops)
    do_constant_folding=True,  # Optimizes constants (e.g., from normalization)
    verbose=False  # Reduce logs; set True for debug
)
app_dir= os.path.join(cwd, "../app/")

In [6]:
def onnx_inference(onnx_path, image_path, transforms, class_names, device):
    try:
        img= Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return
    
    input_tensor= transforms(img)
    input_batch= input_tensor.unsqueeze(0)
    input_batch= input_batch.to(device)

    onnx_input= input_batch.numpy(force= True)
    ort_session= onnxruntime.InferenceSession(onnx_path, providers= ['CPUExecutionProvider'])
    onnx_runtime_input= {'x': onnx_input}
    onnx_runtime_output= ort_session.run(None, onnx_runtime_input)[0]
    onnx_tensor_output= torch.tensor(onnx_runtime_output)
    onnx_probs= F.softmax(onnx_tensor_output, dim= 1)
    onnx_confidence, onnx_pred_idx= torch.max(onnx_probs, 1)
    print(f"Prediction: {class_names[onnx_pred_idx]} | Confidence: {onnx_confidence.item():.4f}")
    

In [7]:
onnx_model_path= os.path.join(artifacts_dir, "resnet18_fft.onnx")
onnx_inference(onnx_model_path, image_path, transform, class_names, device)

Prediction: cockroach | Confidence: 0.5044


In [8]:
label_mapping= {i: class_name for i, class_name in enumerate(class_names)}
label_file_path= os.path.join(artifacts_dir + "/label_mapping.json")
with open(label_file_path, 'w') as f:
    json.dump(label_mapping, f, indent= 4)
    